## Load the necessary library


In [1]:
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from unityagents import UnityEnvironment

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Load the environment

In [2]:
env = UnityEnvironment(file_name='../Reacher_Windows_x86_64/Reacher.exe')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### Method to plot the progress of the agent's score

In [4]:
def plot_result(scores):
    # plot the scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

## Method to see the agent in action

In [22]:
from agents.models_success import Actor
def act(states, actor_network, device):
    ret = None
    actor_network.eval()
    for i in range(len(states)):
        state = torch.from_numpy(states[i]).float().unsqueeze(0).to(device)
        to_add = actor_network(state).cpu().data.numpy()
        if ret is None:
            ret = to_add
        else:
            ret = np.concatenate((ret, to_add))
    return ret

def play(actor_network, device):
    actor_network.eval()
    env_info = env.reset(train_mode=False)[brain_name]
    states = env_info.vector_observations
    scores_one_episode = np.zeros(states.shape[0])
    for _ in range(200):
        actions = act(states, actor_network, device)             # select an action (for each agent)
        env_info = env.step(actions)[brain_name]                 # send all actions to the environment
        next_states = env_info.vector_observations               # get next state (for each agent)
        rewards = env_info.rewards                               # get reward (for each agent)
        dones = env_info.local_done                              # see if episode finished
        scores_one_episode += env_info.rewards                   # update the score (for each agent)
        states = next_states                                     # roll over states to next time step
        if np.any(dones):                                        # exit loop if episode finished
            break
    return scores_one_episode

def load_and_play(path):
    device = "cpu"
    actor = Actor(env_info.vector_observations.shape[1], brain.vector_action_space_size, 0).to(device)
    actor.load_state_dict(torch.load(path))
    return play(actor, "cpu")

In [33]:
env_info = env.reset(train_mode=False)[brain_name]

## Performance of the agent after a training of 200 episodes with 200 timesteps

In [34]:
scores_one_episode = load_and_play("./weights/200/parallel_dpg_actor.pth")

I:\MyDev\Anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


## Performance of the agent after a training of 700 episodes with 200 timesteps

In [31]:
scores_one_episode = load_and_play("./weights/700/parallel_dpg_actor.pth")

I:\MyDev\Anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


## Performance of the agent after a training of 2,200 episodes with 200 timesteps

In [32]:
scores_one_episode = load_and_play("./weights/2200_final_1000_steps/parallel_dpg_actor.pth")

I:\MyDev\Anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
